In [1]:
import pandas as pd
import numpy as np
import json
from sqlalchemy import create_engine

In [2]:
# read csv file
csv_file1 = "./data/low-birth-weight-WA_OR.csv"
data_df = pd.read_csv(csv_file1)
data_df.head()

,Geography,Year,%_of_babies
0,Clackamas County,1998,5.7
1,Clackamas County,1999,5.0
2,Clackamas County,2000,5.5
3,Clackamas County,2001,5.8
4,Clackamas County,2002,5.0


In [3]:
#rename columns
data_df.rename(columns={'Geography':'location', 'Year' : 'year', '%_of_babies': 'percentage_of_babies' }, inplace=True)
data_df.head() 

,location,year,percentage_of_babies
0,Clackamas County,1998,5.7
1,Clackamas County,1999,5.0
2,Clackamas County,2000,5.5
3,Clackamas County,2001,5.8
4,Clackamas County,2002,5.0


In [4]:
# group by counties then years
df1= data_df.groupby(['location', 'year'])
df1.first()

percentage_of_babies
location          year                      
Clackamas County  1998                   5.7
                  1999                   5.0
                  2000                   5.5
                  2001                   5.8
                  2002                   5.0
                  2003                   6.0
                  2004                   5.6
                  2005                   5.1
                  2006                   6.2
                  2007                   5.2
                  2008                   5.9
                  2009                   5.7
                  2010                   6.5
                  2011                   5.6
                  2012                   5.4
                  2013                   6.1
                  2014                   6.2
                  2015                   6.0
Clark County      1998                   5.2
                  1999                   4.5
                  2000                   5.3
                  2001                   4.6
                  2002                   3.9
                  2003                   5.3
                  2004                   5.7
                  2005                   5.1
                  2006                   6.0
                  2007                   6.5
                  2008                   6.2
                  2009                   6.2
...                                      ...
Washington County 2004                   6.2
                  2005                   5.9
                  2006                   6.2
                  2007                   5.9
                  2008                   5.6
                  2009                   6.3
                  2010                   5.6
                  2011                   6.0
                  2012                   6.2
                  2013                   5.8
                  2014                   5.9
                  2015                   6.1
Yamhill County    1998                   5.1
                  1999                   6.1
                  2000                   5.8
                  2001                   5.0
                  2002                   6.4
                  2003                   5.6
                  2004                   5.2
                  2005                   5.9
                  2006                   5.4
                  2007                   5.5
                  2008                   5.0
                  2009                   5.5
                  2010                   5.5
                  2011                   6.5
                  2012                   4.7
                  2013                   6.2
                  2014                   5.4
                  2015                   6.0

[180 rows x 1 columns]

In [5]:
# min, max and mean for percentage_of_babies per location, all years 1998-2015
df1= data_df.groupby(['location']).percentage_of_babies.agg(['min', 'max', 'mean'])
#round mean to one decimal
df2 = df1.round(1)
# rename some columns
df2.rename(columns={'min':'Min', 'max':'Max', 'mean':'Mean'}, inplace=True)
df2

,Min,Max,Mean
location,,,
Clackamas County,5.0,6.5,5.7
Clark County,3.9,6.5,5.6
Columbia County,3.4,8.2,5.6
Multnomah County,5.3,7.4,6.3
Oregon,5.4,6.4,6.0
Skamania County,0.8,11.1,6.0
United States,7.6,8.3,8.0
Washington,5.6,6.5,6.1
Washington County,5.2,6.3,5.9


In [6]:
# add Array and Array minus needed for error bars chart
df2['ArrayPlus'] = df2['Max'] - df2['Mean']
df2['ArrayMinus'] = df2['Mean'] - df2['Min']
df_aggregates = df2.round(1)
df_aggregates

,Min,Max,Mean,ArrayPlus,ArrayMinus
location,,,,,
Clackamas County,5.0,6.5,5.7,0.8,0.7
Clark County,3.9,6.5,5.6,0.9,1.7
Columbia County,3.4,8.2,5.6,2.6,2.2
Multnomah County,5.3,7.4,6.3,1.1,1.0
Oregon,5.4,6.4,6.0,0.4,0.6
Skamania County,0.8,11.1,6.0,5.1,5.2
United States,7.6,8.3,8.0,0.3,0.4
Washington,5.6,6.5,6.1,0.4,0.5
Washington County,5.2,6.3,5.9,0.4,0.7


In [7]:
# connect to local database (you need first to create "low_birth_weight" database)
# update with your localhost, user and paswword data for postgres
rds_connection_string = "postgres:postgres@localhost:5432/low_birth_weight"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [8]:
# check for tables
engine.table_names()

[]

In [9]:
# create postgres table for birth data
data_df.to_sql(name='birth_data', con=engine)
#data_df.to_sql(name='birth_data', con=engine, if_exists='append')

# run only one time this command; second time will give an error because the table already exist so you
# have to delete first the table from Postgres

In [10]:
# read birth_data table
pd.read_sql_query('select * from birth_data', con=engine).head()

,index,location,year,percentage_of_babies
0,0,Clackamas County,1998,5.7
1,1,Clackamas County,1999,5.0
2,2,Clackamas County,2000,5.5
3,3,Clackamas County,2001,5.8
4,4,Clackamas County,2002,5.0


In [11]:
# create table for aggregate data

df_aggregates.to_sql(name='aggregate_data', con=engine)
# df_aggregates.to_sql(name='aggregate_data', con=engine, if_exists='append')

# run only one time this command; second time will give an error because the table already exist so you
# have to delete first the table from Postgres

In [12]:
# read aggregate_data table
pd.read_sql_query('select * from aggregate_data', con=engine).head()

,location,Min,Max,Mean,ArrayPlus,ArrayMinus
0,Clackamas County,5.0,6.5,5.7,0.8,0.7
1,Clark County,3.9,6.5,5.6,0.9,1.7
2,Columbia County,3.4,8.2,5.6,2.6,2.2
3,Multnomah County,5.3,7.4,6.3,1.1,1.0
4,Oregon,5.4,6.4,6.0,0.4,0.6


In [13]:
engine.table_names()

['birth_data', 'aggregate_data']

In [14]:
# # read json file
# file = './data/oregon-washignton-counties-geojson.json'

# with open(file, encoding='utf-8-sig') as json_file:
# #     json_data = json.loads(text)
#     json_data = json.load(json_file)
#     print(json_data)

{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {'GEO_ID': '0500000US41005', 'STATE': '41', 'COUNTY': '005', 'NAME': 'Clackamas', 'LSAD': 'County', 'CENSUSAREA': 1870.322}, 'geometry': {'type': 'Polygon', 'coordinates': [[[-122.438075, 45.461145], [-122.421827, 45.461435], [-122.414457, 45.461579], [-122.41397, 45.461579], [-122.389346, 45.4613], [-122.38269, 45.461273], [-122.372796, 45.46123], [-122.348317, 45.461505], [-122.31608, 45.461542], [-122.305836, 45.461439], [-122.273126, 45.461491], [-122.264032, 45.461537], [-122.16663, 45.461124], [-122.086491, 45.460364], [-122.086484, 45.461082], [-122.004262, 45.461589], [-121.983247, 45.461462], [-121.982599, 45.460231], [-121.822, 45.460222], [-121.821994, 45.461674], [-121.820394, 45.461666], [-121.815598, 45.459072], [-121.812723, 45.456147], [-121.806848, 45.454326], [-121.80578, 45.452734], [-121.807236, 45.449389], [-121.804788, 45.444675], [-121.806679, 45.442732], [-121.805557, 45.438175], [-121.